In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
MODEL_DIR = './model'

In [3]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

def maybe_download():
  train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
  test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

  return train_path, test_path

def load_data(y_name='Species'):
  """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
  import pandas as pd
  train_path, test_path = maybe_download()

  train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
  train_x, train_y = train, train.pop(y_name)

  test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
  test_x, test_y = test, test.pop(y_name)

  return (train_x, train_y), (test_x, test_y)

In [4]:
def make_train_input_fn(features, labels, batch_size):
  def train_input_fn():
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    
    # Return the dataset.
    return dataset
  return train_input_fn

In [5]:
# Fetch the data
(train_x, train_y), (test_x, test_y) = load_data()

# Feature columns describe how to use the input.
my_feature_columns = []
for key in train_x.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [6]:
! rm -rf $MODEL_DIR

In [7]:
my_checkpointing_config = tf.estimator.RunConfig(
    save_checkpoints_steps = 2000,
    #keep_checkpoint_max = 10,
    log_step_count_steps = 500
)


# Build 2 hidden layer DNN with 10, 10 units respectively.
classifier = tf.estimator.DNNClassifier(
  feature_columns=my_feature_columns,
  # Two hidden layers of 10 nodes each.
  hidden_units=[10, 10],
  # The model must choose between 3 classes.
  n_classes=3,
  model_dir=MODEL_DIR,
  #config=my_checkpointing_config
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9f3323ea10>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': './model', '_global_id_in_cluster': 0, '_save_summary_steps': 100}


In [8]:
# Train the Model.
classifier.train(
  input_fn=make_train_input_fn(train_x, train_y, 100),
  steps=20000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./model/model.ckpt.
INFO:tensorflow:loss = 240.07687, step = 1
INFO:tensorflow:global_step/sec: 447.698
INFO:tensorflow:loss = 13.670153, step = 101 (0.225 sec)
INFO:tensorflow:global_step/sec: 627.877
INFO:tensorflow:loss = 10.7018585, step = 201 (0.160 sec)
INFO:tensorflow:global_step/sec: 645.987
INFO:tensorflow:loss = 6.5004654, step = 301 (0.155 sec)
INFO:tensorflow:global_step/sec: 619.897
INFO:tensorflow:loss = 4.8978486, step = 401 (0.160 sec)
INFO:tensorflow:global_step/sec: 642.294
INFO:tensorflow:loss = 7.415059, step = 501 (0.156 sec)
INFO:tensorflow:global_step/sec: 674.832
INFO:tensorflow:loss = 5.6658945, step = 601 (0.148 sec)
INFO:tensorflow:global_step/sec: 706.823
INFO:tensorflow:loss = 4.

INFO:tensorflow:global_step/sec: 738.448
INFO:tensorflow:loss = 2.1033885, step = 8101 (0.134 sec)
INFO:tensorflow:global_step/sec: 735.44
INFO:tensorflow:loss = 2.4017127, step = 8201 (0.136 sec)
INFO:tensorflow:global_step/sec: 730.551
INFO:tensorflow:loss = 3.0140357, step = 8301 (0.137 sec)
INFO:tensorflow:global_step/sec: 729.348
INFO:tensorflow:loss = 1.2345229, step = 8401 (0.137 sec)
INFO:tensorflow:global_step/sec: 675.342
INFO:tensorflow:loss = 3.2531111, step = 8501 (0.148 sec)
INFO:tensorflow:global_step/sec: 746.219
INFO:tensorflow:loss = 2.3840756, step = 8601 (0.133 sec)
INFO:tensorflow:global_step/sec: 772.069
INFO:tensorflow:loss = 1.9319904, step = 8701 (0.130 sec)
INFO:tensorflow:global_step/sec: 718.887
INFO:tensorflow:loss = 1.8156391, step = 8801 (0.139 sec)
INFO:tensorflow:global_step/sec: 698.709
INFO:tensorflow:loss = 0.9036726, step = 8901 (0.142 sec)
INFO:tensorflow:global_step/sec: 760.775
INFO:tensorflow:loss = 2.174258, step = 9001 (0.131 sec)
INFO:tensorf

INFO:tensorflow:loss = 1.0387913, step = 16301 (0.133 sec)
INFO:tensorflow:global_step/sec: 691.4
INFO:tensorflow:loss = 0.6569538, step = 16401 (0.144 sec)
INFO:tensorflow:global_step/sec: 745.624
INFO:tensorflow:loss = 0.6468867, step = 16501 (0.134 sec)
INFO:tensorflow:global_step/sec: 756.304
INFO:tensorflow:loss = 0.62901366, step = 16601 (0.133 sec)
INFO:tensorflow:global_step/sec: 743.964
INFO:tensorflow:loss = 1.0946182, step = 16701 (0.134 sec)
INFO:tensorflow:global_step/sec: 737.197
INFO:tensorflow:loss = 0.66482663, step = 16801 (0.136 sec)
INFO:tensorflow:global_step/sec: 719.595
INFO:tensorflow:loss = 0.76485705, step = 16901 (0.140 sec)
INFO:tensorflow:global_step/sec: 747.86
INFO:tensorflow:loss = 0.81899536, step = 17001 (0.133 sec)
INFO:tensorflow:global_step/sec: 760.989
INFO:tensorflow:loss = 0.72281986, step = 17101 (0.131 sec)
INFO:tensorflow:global_step/sec: 763.207
INFO:tensorflow:loss = 0.5403791, step = 17201 (0.132 sec)
INFO:tensorflow:global_step/sec: 768.48

In [9]:
def make_eval_input_fn(features, labels=None, batch_size=100):
  def eval_input_fn():
    """An input function for evaluation or prediction"""
    if labels is None:
        # No labels, use only features.
        inputs = dict(features)
    else:
        inputs = (dict(features), labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset
  
  return eval_input_fn

In [10]:
# Evaluate the model.
eval_result = classifier.evaluate(
  input_fn=make_eval_input_fn(test_x, test_y))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-11-17:37:01
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-11-17:37:02
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.96666664, average_loss = 0.29259145, global_step = 20000, loss = 8.777743

Test set accuracy: 0.967



In [11]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
  'SepalLength': [5.1, 5.9, 6.9],
  'SepalWidth': [3.3, 3.0, 3.1],
  'PetalLength': [1.7, 4.2, 5.4],
  'PetalWidth': [0.5, 1.5, 2.1],
}

predictions = classifier.predict(input_fn=make_eval_input_fn(predict_x))

template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

for pred_dict, expec in zip(predictions, expected):
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

  print(template.format(SPECIES[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "Setosa" (99.9%), expected "Setosa"

Prediction is "Versicolor" (100.0%), expected "Versicolor"

Prediction is "Virginica" (100.0%), expected "Virginica"
